# CSCN8010 Foundations of Machine Learning Frameworks 
## Practical Lab 3 : Vanilla CNN and Fine-Tune VGG16 - for Dogs and Cats Classification

### Submitted by
- Name : Sreehari Prathap
- Student ID : 8903199
- Email : sprathap3199@conestogac.on.ca

In [ ]:
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
import seaborn as sns


In [ ]:
data_folder = pathlib.Path('../CSCN8010/data/kaggle_dogs_vs_cats_small')

train_dataset = image_dataset_from_directory(
    data_folder / "train",
    image_size=(180, 180),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    data_folder / "validation",
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    data_folder / "test",
    image_size=(180, 180),
    batch_size=32
)


In [ ]:
# Visualizing sample images
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")


In [ ]:
# Display class distribution
class_counts = {}
for _, labels in train_dataset.unbatch():
    for label in labels:
        class_counts[int(label)] = class_counts.get(int(label), 0) + 1

sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.title("Class Distribution")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()


In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./models/convnet_scratch.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    callbacks=callbacks
)


In [ ]:
conv_base = keras.applications.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3)
)

# Freeze all layers except the last few
conv_base.trainable = False

# Build the model
inputs = keras.Input(shape=(180, 180, 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_vgg16 = keras.Model(inputs, outputs)

model_vgg16.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./models/vgg16_fine_tuned.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

history_vgg16 = model_vgg16.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    callbacks=callbacks
)


In [ ]:
best_model_custom = keras.models.load_model("./models/convnet_scratch.keras")
best_model_vgg16 = keras.models.load_model("./models/vgg16_fine_tuned.keras")


In [ ]:
# Evaluate models
for model, name in zip([best_model_custom, best_model_vgg16], ["Custom CNN", "VGG16"]):
    print(f"Evaluating {name}")
    test_loss, test_acc = model.evaluate(test_dataset)
    print(f"Test Accuracy: {test_acc:.2f}")

    y_true = np.concatenate([y for _, y in test_dataset], axis=0)
    y_pred = model.predict(test_dataset).ravel()
    y_pred_classes = (y_pred > 0.5).astype(int)

    print(f"Confusion Matrix for {name}")
    cm = confusion_matrix(y_true, y_pred_classes)
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title(f"Confusion Matrix: {name}")
    plt.show()

    print(f"Classification Report for {name}")
    print(classification_report(y_true, y_pred_classes))

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    plt.plot(recall, precision, label=name)
    plt.title("Precision-Recall Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.show()
